# Importing the Libraries

In [1]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.utils import shuffle
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
import os
import pandas as pd
import librosa
import glob 
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

# Loading the dataset

In [2]:
rav_sav= os.listdir('Dataset1/')
tess= os.listdir('Dataset2/')

# Printing size of dataset

In [3]:
print(len(rav_sav))
print(len(tess))

1920
2800


# Mapping the data of RAVDESS and SAVEE

In [4]:
emotions=[]
for file in rav_sav:
    if file[6:-16]=='01':
        emotions.append('neutral')
    elif file[6:-16]=='03' and int(file[18:-4])%2==0:
        emotions.append('happy')
    elif file[6:-16]=='03' and int(file[18:-4])%2==1:
        emotions.append('happy')
    elif file[6:-16]=='04' and int(file[18:-4])%2==0:
        emotions.append('sad')
    elif file[6:-16]=='04' and int(file[18:-4])%2==1:
        emotions.append('sad')
    elif file[6:-16]=='05' and int(file[18:-4])%2==0:
        emotions.append('angry')
    elif file[6:-16]=='05' and int(file[18:-4])%2==1:
        emotions.append('angry')
    elif file[6:-16]=='06' and int(file[18:-4])%2==0:
        emotions.append('fearful')
    elif file[6:-16]=='06' and int(file[18:-4])%2==1:
        emotions.append('fearful')
    elif file[6:-16]=='07':
        emotions.append('disgust')
    elif file[:1]=='a':
        emotions.append('angry')
    elif file[:1]=='f':
        emotions.append('fearful')
    elif file[:1]=='h':
        emotions.append('happy')
    elif file[:2]=='sa':
        emotions.append('sad')

# Mapping the data of TESS

In [5]:
for file in tess:
    if file[-11:-4]=='neutral':
        emotions.append('neutral')
    elif file[-9:-4]=='angry':
        emotions.append('angry')
    elif file[-11:-4]=='disgust':
        emotions.append('disgust')
    elif file[-9:-4]=='happy':
        emotions.append('happy')
    elif file[-7:-4]=='sad':
        emotions.append('sad')
    elif file[-8:-4]=='fear':
        emotions.append('fearful')

# Labelling the data

In [6]:
labels = pd.DataFrame(emotions)

# Feature Extraction of RAVDESS and SAVEE dataset using mfcc

In [7]:
df = pd.DataFrame(columns=['feature'])
count=0
for i,y in enumerate(rav_sav):
    if rav_sav[i][6:-16]!='02' and rav_sav[i][6:-16]!='08' and rav_sav[i][:2]!='su' and rav_sav[i][:1]!='n' and rav_sav[i][:1]!='d':
        X, sample_rate = librosa.load('Dataset1/'+y, res_type='kaiser_fast',duration=2.5,sr=22050,offset=0)
        sample_rate = np.array(sample_rate)
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13), axis=0)
        feature = mfccs
        df.loc[count] = [feature]
        count=count+1  

# Feature extraction of TESS dataset using mfcc

In [8]:
for i,y in enumerate(tess):
     if tess[i][-6:-4]!='ps':
        X, sample_rate = librosa.load('Dataset2/'+y, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0)
        sample_rate = np.array(sample_rate)
        mfccs = np.mean(librosa.feature.mfcc(y=X,  sr=sample_rate,  n_mfcc=13), axis=0)
        feature = mfccs
        df.loc[count] = [feature]
        count=count+1 

In [9]:
print(df)

                                                feature
0     [-65.946724, -65.946724, -65.946724, -65.94672...
1     [-63.72445, -63.72445, -63.72445, -63.72445, -...
2     [-59.59301, -59.59301, -59.59301, -59.59301, -...
3     [-66.73365, -66.73365, -66.73365, -66.73365, -...
4     [-61.566444, -57.94428, -56.239346, -58.175125...
...                                                 ...
3691  [-43.600605, -43.108433, -42.603493, -43.46729...
3692  [-31.2706, -23.65622, -21.566856, -22.070164, ...
3693  [-32.686943, -30.507328, -27.698042, -31.79878...
3694  [-41.914738, -40.3994, -39.37127, -39.626366, ...
3695  [-33.994755, -35.7186, -36.789467, -34.117706,...

[3696 rows x 1 columns]


# Splitting into train and test data

In [10]:
tempdf = pd.DataFrame(df['feature'].values.tolist())
df_new = pd.concat([tempdf,labels], axis=1)

In [11]:
ren_df = df_new.rename(index=str, columns={"0": "label"})
ren_df = shuffle(df_new)
ren_df = ren_df.fillna(0)
final_df = np.random.rand(len(ren_df)) < 0.8

In [12]:
train = ren_df[final_df]
test = ren_df[~final_df]
print(len(train))
print(len(test))

2964
732


In [13]:
print(tempdf)

            0          1          2          3          4          5    \
0    -65.946724 -65.946724 -65.946724 -65.946724 -65.946724 -65.946724   
1    -63.724449 -63.724449 -63.724449 -63.724449 -63.724449 -63.724449   
2    -59.593010 -59.593010 -59.593010 -59.593010 -59.593010 -59.593010   
3    -66.733650 -66.733650 -66.733650 -66.733650 -66.733650 -66.733650   
4    -61.566444 -57.944279 -56.239346 -58.175125 -58.332890 -60.323990   
...         ...        ...        ...        ...        ...        ...   
3691 -43.600605 -43.108433 -42.603493 -43.467293 -41.499535 -41.084091   
3692 -31.270599 -23.656219 -21.566856 -22.070164 -24.841099 -28.609322   
3693 -32.686943 -30.507328 -27.698042 -31.798786 -33.790195 -32.447849   
3694 -41.914738 -40.399399 -39.371269 -39.626366 -35.696751 -28.868685   
3695 -33.994755 -35.718601 -36.789467 -34.117706 -30.381868 -28.359896   

            6          7          8          9    ...        206        207  \
0    -65.946724 -65.752853 -65.9

In [14]:
train_set = train.iloc[:, :-1]
train_label = train.iloc[:, -1:]
test_set = test.iloc[:, :-1]
test_label = test.iloc[:, -1:]

# Getting train and test data

In [15]:
X_train = np.array(train_set)
y_train = np.array(train_label)
X_test = np.array(test_set)
y_test = np.array(test_label)

lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [16]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(2964, 216)
(2964, 6)
(732, 216)
(732, 6)


In [17]:
y_train1d = []
for i in range(len(y_train)):
    for j in range(len(y_train[0])):
        if(y_train[i][j] == 1):
            y_train1d.append(j+1)

In [18]:
print(y_train1d)

[4, 1, 4, 5, 4, 4, 3, 2, 2, 1, 2, 6, 6, 5, 3, 5, 6, 1, 6, 5, 4, 3, 1, 4, 2, 2, 1, 4, 2, 5, 5, 6, 2, 3, 5, 5, 5, 5, 2, 3, 1, 6, 6, 6, 3, 6, 2, 3, 1, 3, 5, 6, 4, 2, 6, 1, 6, 5, 6, 4, 3, 1, 3, 1, 3, 5, 3, 6, 4, 5, 5, 4, 2, 3, 5, 1, 3, 5, 5, 6, 1, 3, 4, 2, 4, 5, 5, 4, 1, 2, 1, 1, 3, 1, 3, 1, 6, 6, 3, 4, 3, 2, 3, 4, 2, 1, 1, 2, 2, 2, 6, 2, 1, 1, 6, 3, 1, 4, 3, 5, 6, 1, 2, 3, 5, 3, 4, 2, 2, 1, 4, 1, 6, 2, 1, 3, 2, 6, 2, 3, 6, 4, 5, 6, 1, 3, 1, 1, 4, 2, 1, 1, 3, 5, 2, 4, 5, 3, 2, 1, 3, 3, 4, 6, 5, 3, 6, 6, 4, 2, 4, 5, 2, 5, 4, 4, 1, 1, 2, 6, 5, 2, 3, 1, 1, 2, 2, 1, 4, 1, 4, 1, 6, 5, 6, 1, 5, 4, 3, 5, 3, 6, 6, 3, 1, 3, 2, 2, 3, 2, 5, 6, 1, 3, 2, 6, 5, 4, 3, 1, 1, 1, 4, 3, 6, 6, 5, 3, 5, 4, 2, 3, 1, 3, 3, 6, 3, 2, 3, 6, 5, 4, 2, 5, 1, 3, 5, 1, 1, 1, 1, 2, 6, 3, 1, 1, 3, 5, 1, 4, 5, 6, 4, 2, 6, 6, 5, 4, 4, 3, 5, 2, 3, 5, 4, 5, 5, 6, 2, 4, 1, 4, 4, 4, 1, 2, 5, 3, 4, 4, 6, 4, 1, 2, 5, 6, 3, 1, 4, 5, 3, 4, 1, 3, 3, 2, 4, 5, 5, 2, 2, 3, 6, 3, 5, 3, 4, 4, 4, 1, 2, 6, 2, 2, 4, 5, 3, 5, 6, 3, 2, 5, 1, 

In [19]:
y_test1d = []
for i in range(len(y_test)):
    for j in range(len(y_test[0])):
        if(y_test[i][j] == 1):
            y_test1d.append(j+1)

In [20]:
print(y_test1d)

[2, 6, 2, 6, 3, 6, 1, 4, 6, 2, 2, 4, 6, 5, 6, 3, 1, 5, 1, 1, 6, 1, 6, 6, 2, 3, 4, 6, 1, 1, 3, 6, 5, 6, 1, 1, 1, 6, 4, 3, 6, 1, 3, 4, 5, 4, 2, 4, 4, 4, 6, 6, 4, 4, 3, 4, 1, 4, 4, 6, 3, 6, 3, 2, 3, 6, 4, 1, 4, 6, 3, 1, 2, 6, 4, 1, 3, 2, 3, 6, 6, 3, 6, 1, 5, 6, 6, 1, 3, 2, 5, 3, 4, 5, 3, 5, 2, 3, 5, 4, 2, 1, 6, 2, 2, 3, 6, 3, 4, 2, 3, 2, 4, 4, 3, 1, 3, 5, 3, 1, 6, 2, 2, 6, 5, 2, 3, 3, 2, 6, 6, 6, 2, 4, 2, 3, 4, 3, 3, 2, 6, 3, 1, 5, 3, 5, 1, 3, 6, 2, 3, 4, 4, 1, 6, 5, 6, 1, 2, 3, 5, 1, 5, 6, 3, 5, 2, 5, 1, 5, 1, 6, 3, 4, 1, 2, 1, 6, 5, 2, 4, 4, 6, 2, 4, 4, 1, 3, 5, 6, 1, 1, 4, 1, 5, 2, 1, 4, 2, 2, 1, 6, 2, 6, 2, 4, 5, 3, 4, 1, 2, 3, 3, 1, 5, 3, 2, 6, 2, 5, 1, 5, 2, 6, 3, 4, 4, 2, 2, 6, 3, 3, 3, 2, 4, 1, 6, 2, 5, 1, 1, 4, 5, 4, 1, 2, 2, 5, 3, 5, 3, 6, 1, 4, 4, 5, 1, 3, 4, 3, 2, 4, 5, 5, 3, 6, 4, 6, 1, 6, 5, 6, 6, 3, 3, 1, 6, 1, 6, 1, 2, 4, 1, 1, 2, 1, 2, 4, 1, 5, 6, 4, 6, 3, 1, 1, 1, 3, 5, 1, 6, 5, 2, 2, 4, 6, 4, 6, 6, 2, 6, 6, 4, 2, 3, 2, 2, 3, 4, 3, 2, 6, 2, 4, 6, 1, 2, 1, 1, 2, 3, 3, 6, 

In [21]:
print(X_train)

[[-35.41674042 -34.12383652 -33.70621109 ...   0.           0.
    0.        ]
 [-31.68978119 -31.47378922 -31.38124657 ...   0.           0.
    0.        ]
 [-42.22000504 -40.52980042 -38.66399765 ...   0.           0.
    0.        ]
 ...
 [-40.69172668 -38.9319725  -39.172966   ...   0.           0.
    0.        ]
 [-51.48413849 -50.75852966 -48.6944809  ...   0.           0.
    0.        ]
 [-46.25503922 -46.23346329 -46.34589005 ...   0.           0.
    0.        ]]


# Feature Scaling

In [22]:
from sklearn.preprocessing import StandardScaler  
sc = StandardScaler()  
X_train = sc.fit_transform(X_train)  
X_test = sc.transform(X_test)  

# Training the model using naive bayes classifier

In [24]:
from sklearn.naive_bayes import GaussianNB  
classifier = GaussianNB()  
classifier.fit(X_train, y_train1d)  

GaussianNB()

# Predicting 

In [25]:
y_pred = classifier.predict(X_test)  

In [27]:
print(accuracy_score(y_test1d, y_pred))

0.32786885245901637
